# PyAndHold - Portfolio Analysis Examples

This notebook provides clean, simple examples of portfolio optimization and analysis.

**Structure:**
1. Imports
2. Configuration (portfolios to test)
3. Analysis outputs (charts & tables)
4. Optimization examples

## 1. Imports

In [ ]:
# Core imports
import pandas as pd
import numpy as np
from datetime import datetime

# PyAndHold imports
from pyandhold import Portfolio, DataDownloader, PortfolioOptimizer
from pyandhold.visualization import PortfolioVisualizer

## 2. Configuration

Define portfolios to analyze. Comment out portfolios you don't want to test.

In [ ]:
# Portfolio 1: Equal Weight Tech Portfolio
tech_weights = {
    'AAPL': 0.25,
    'MSFT': 0.25,
    'GOOGL': 0.25,
    'AMZN': 0.25
}

# Portfolio 2: Diversified Portfolio
diversified_weights = {
    'AAPL': 0.15,
    'MSFT': 0.15,
    'JPM': 0.10,
    'JNJ': 0.10,
    'GLD': 0.15,
    'TLT': 0.20,
    'VNQ': 0.15
}

# Portfolio 3: 60/40 Classic
classic_60_40 = {
    'SPY': 0.60,  # S&P 500
    'TLT': 0.40   # 20+ Year Treasury Bonds
}

# Common parameters
start_date = '2020-01-01'
end_date = '2023-12-31'
initial_capital = 100000
rebalance_freq = 'Q'  # Q=Quarterly, M=Monthly, Y=Yearly, None=Buy&Hold

## 3. Analysis & Visualizations

Choose which portfolio to analyze and what outputs to generate.

In [ ]:
# Select portfolio to analyze
weights_to_analyze = tech_weights  # Change this to test different portfolios

# Create portfolio
portfolio = Portfolio(
    weights=weights_to_analyze,
    start_date=start_date,
    end_date=end_date,
    initial_capital=initial_capital,
    rebalance_frequency=rebalance_freq
)

# Calculate metrics
metrics = portfolio.calculate_metrics()
portfolio_value = portfolio.calculate_portfolio_value()

print("Portfolio Metrics:")
print(f"  Total Return: {metrics['total_return']:.2%}")
print(f"  Annualized Return: {metrics['annualized_return']:.2%}")
print(f"  Volatility: {metrics['volatility']:.2%}")
print(f"  Sharpe Ratio: {metrics['sharpe_ratio']:.3f}")
print(f"  Max Drawdown: {metrics['max_drawdown']:.2%}")
print(f"  Sortino Ratio: {metrics['sortino_ratio']:.3f}")

### 3.1 Performance Chart

In [ ]:
visualizer = PortfolioVisualizer()

# Portfolio performance over time
perf_fig = visualizer.plot_performance(
    portfolio_value,
    title="Portfolio Value Over Time"
)
perf_fig.show()

### 3.2 Returns Distribution

In [ ]:
# Returns distribution histogram
dist_fig = visualizer.plot_returns_distribution(
    portfolio.portfolio_returns,
    title="Daily Returns Distribution"
)
dist_fig.show()

### 3.3 Drawdown Chart

In [ ]:
# Drawdown over time
dd_fig = visualizer.plot_drawdown(
    portfolio_value,
    title="Portfolio Drawdown"
)
dd_fig.show()

### 3.4 Portfolio Weights

In [ ]:
# Pie chart of weights
pie_fig = visualizer.plot_weights_pie(
    weights_to_analyze,
    title="Portfolio Allocation"
)
pie_fig.show()

# Table of weights
weights_df = pd.DataFrame([
    {"Ticker": ticker, "Weight": f"{weight*100:.2f}%"}
    for ticker, weight in sorted(weights_to_analyze.items(), key=lambda x: x[1], reverse=True)
])
display(weights_df)

### 3.5 Correlation Heatmap

In [ ]:
# Asset correlation
corr_matrix = portfolio.get_correlation_matrix()
corr_fig = visualizer.plot_correlation_heatmap(
    corr_matrix,
    title="Asset Correlations"
)
corr_fig.show()

### 3.6 Rolling Metrics (Optional)

In [ ]:
# Rolling Sharpe Ratio (63-day window ~ 3 months)
window = 63
returns = portfolio.portfolio_returns

if len(returns) >= window:
    rolling_mean = returns.rolling(window).mean()
    rolling_std = returns.rolling(window).std()
    rolling_sharpe = rolling_mean / rolling_std * np.sqrt(252)
    
    sharpe_fig = visualizer.plot_single_metric_timeseries(
        rolling_sharpe,
        title="63-Day Rolling Sharpe Ratio",
        y_label="Sharpe Ratio",
        color='green',
        show_mean=True
    )
    sharpe_fig.show()

## 4. Portfolio Optimization

Optimize portfolios using different strategies. Uncomment the ones you want to test.

In [ ]:
# Download returns data for optimization
downloader = DataDownloader()

# Define asset universe
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'JPM', 'JNJ', 'GLD', 'TLT']

returns = downloader.download_returns(
    tickers,
    start_date=start_date,
    end_date=end_date
)

# Initialize optimizer
optimizer = PortfolioOptimizer(returns)

### 4.1 Maximum Sharpe Ratio

In [ ]:
# Optimize for maximum Sharpe ratio
max_sharpe_weights = optimizer.optimize_sharpe(
    weight_bounds=(0.0, 0.4)  # Min 0%, Max 40% per asset
)

print("Maximum Sharpe Ratio Portfolio:")
for ticker, weight in sorted(max_sharpe_weights.items(), key=lambda x: x[1], reverse=True):
    if weight > 0.01:  # Only show weights > 1%
        print(f"  {ticker}: {weight:.2%}")

### 4.2 Minimum Variance

In [ ]:
# Optimize for minimum variance (lowest risk)
min_var_weights = optimizer.optimize_min_variance(
    weight_bounds=(0.0, 1.0)
)

print("Minimum Variance Portfolio:")
for ticker, weight in sorted(min_var_weights.items(), key=lambda x: x[1], reverse=True):
    if weight > 0.01:
        print(f"  {ticker}: {weight:.2%}")

### 4.3 Risk Parity

In [ ]:
# Risk parity - equal risk contribution from each asset
risk_parity_weights = optimizer.optimize_risk_parity(
    weight_bounds=(0.0, 0.5)
)

print("Risk Parity Portfolio:")
for ticker, weight in sorted(risk_parity_weights.items(), key=lambda x: x[1], reverse=True):
    if weight > 0.01:
        print(f"  {ticker}: {weight:.2%}")

### 4.4 Maximum Return with Volatility Constraint

In [ ]:
# # Optimize for maximum return with volatility limit
# max_return_weights = optimizer.optimize_max_return(
#     max_volatility=0.20,  # Max 20% annual volatility
#     weight_bounds=(0.0, 1.0)
# )

# print("Maximum Return Portfolio (vol ≤ 20%):")
# for ticker, weight in sorted(max_return_weights.items(), key=lambda x: x[1], reverse=True):
#     if weight > 0.01:
#         print(f"  {ticker}: {weight:.2%}")

### 4.5 Efficient Frontier

In [ ]:
# Calculate efficient frontier
frontier = optimizer.efficient_frontier(
    n_portfolios=30,
    weight_bounds=(0.0, 1.0)
)

# Plot efficient frontier
ef_fig = visualizer.plot_efficient_frontier(frontier)
ef_fig.show()

### 4.6 Compare Multiple Strategies

In [ ]:
# Create portfolios with different optimization strategies
strategies = {
    'Equal Weight': {ticker: 1/len(tickers) for ticker in tickers},
    'Max Sharpe': max_sharpe_weights,
    'Min Variance': min_var_weights,
    'Risk Parity': risk_parity_weights
}

# Calculate performance for each
comparison_results = []

for strategy_name, weights in strategies.items():
    p = Portfolio(
        weights=weights,
        start_date=start_date,
        end_date=end_date,
        initial_capital=100000
    )
    m = p.calculate_metrics()
    
    comparison_results.append({
        'Strategy': strategy_name,
        'Return': f"{m['annualized_return']:.2%}",
        'Volatility': f"{m['volatility']:.2%}",
        'Sharpe': f"{m['sharpe_ratio']:.3f}",
        'Max DD': f"{m['max_drawdown']:.2%}"
    })

comparison_df = pd.DataFrame(comparison_results)
print("\nStrategy Comparison:")
display(comparison_df)

### 4.7 Multi-Strategy Performance Chart

In [ ]:
# Compare performance visually
import plotly.graph_objects as go

fig = go.Figure()
colors = ['blue', 'red', 'green', 'purple']

for i, (strategy_name, weights) in enumerate(strategies.items()):
    p = Portfolio(
        weights=weights,
        start_date=start_date,
        end_date=end_date,
        initial_capital=100000
    )
    portfolio_values = p.calculate_portfolio_value()
    
    fig.add_trace(go.Scatter(
        x=portfolio_values.index,
        y=portfolio_values.values,
        mode='lines',
        name=strategy_name,
        line=dict(color=colors[i], width=2)
    ))

fig.update_layout(
    title="Strategy Performance Comparison",
    xaxis_title="Date",
    yaxis_title="Portfolio Value ($)",
    hovermode='x unified',
    width=1000,
    height=600
)
fig.show()

## Summary

This notebook demonstrates:
- Loading and configuring portfolios
- Analyzing portfolio performance with multiple visualizations
- Optimizing portfolios using various strategies
- Comparing different approaches

**Next Steps:**
- Modify the configuration section to test your own portfolios
- Comment/uncomment visualization blocks as needed
- Experiment with different optimization constraints
- Add custom tickers and date ranges